In [ ]:
import basic
import Hydrographs

In [ ]:

ml = basic.load_model(path = 'temp/June2020')
run_name = 'June2020'
info, swr_info, sfr_info, riv_keys_info = basic.load_params(run_name)

datestart = info['start_date']
numdays = info['numdays']
name = info['name']

out_folder = basic.out_folder(run_name)

print(datestart)
print(out_folder)

wells_mod = Hydrographs.load_wells_mod(ml)

obsall = Hydrographs.do_hydros(ml, wells_mod, out_folder, datestart, numdays, skip_plotting = True)
# Hydrographs.plot_one_to_one(obsall, out_folder, byzone=True)



In [ ]:
def plot_residual(allobs, out_folder, ml):

    locs = allobs.drop_duplicates('well').loc[:,['geometry','well']]
    
    allobs.loc[:,'residual'] = allobs.loc[:, 'Observed'] - allobs.loc[:,'Simulated']
    
    allobs = allobs.drop(columns = 'geometry').groupby('well').mean().reset_index()
    
    
    merged = pd.merge(allobs, locs, on = 'well')
    merged = gpd.GeoDataFrame(merged, geometry = 'geometry', crs = 2226)
    
    fig, ax = basic.map_river(m = ml,add_basemap=True)
    ph.remove_ctx_annotations(ax)
    # fig, ax = basic.swr_map(m = ml)
    # fig, ax = basic.basic_map(m = ml)
    basic.set_bounds(ax)
    
    merged.plot('residual', markersize = 15, edgecolor = 'k', ax = ax, legend = True,
                cmap = 'jet', legend_kwds = {'title':'Residual (Obs. - Sim.)', 'loc':'upper left',
                                            'bbox_to_anchor':(1,1)},
              classification_kwds = {'bins':np.arange(-25,26,5)},
                scheme  = 'UserDefined', zorder = 105)

    ph.label_points(ax = ax, gdf = merged,fmt = ".1f",  colname = 'residual',basin_name=None)
    
    ax.text(1,0, 'Labels are residuals',ha = 'right',  transform = ax.transAxes)
    
    plt.savefig(os.path.join(out_folder, 'residuals.png'), dpi=250, bbox_inches='tight')

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import conda_scripts.plot_help as ph
import matplotlib.pyplot as plt
import os

In [ ]:
plot_residual(obsall, out_folder=out_folder, ml = ml)